# Model that work along to dialogflow to get more accuracy and precission

<img src="chat.png">

In [ ]:
import numpy as np
import pandas as pd
import sys
import json
import random
import os,inspect
import re
import string
import h5py
import math as mt
from tqdm import tqdm
import keras.backend as K

import tensorflow as tf
import numpy as np
import os,inspect

from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dropout, Add
from keras.layers import TimeDistributed
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.embeddings import Embedding

from keras.utils import to_categorical
from keras.utils import plot_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import load_model, Model

from IPython.display import SVG
import matplotlib.pyplot as plt
from tqdm import tqdm

from attention_decoder_f import attention_LSTM
%matplotlib inline

# Preprocessing

In [ ]:
# root path
path_file = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))

In [ ]:
# Clean memory gpu
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [ ]:
limit_mem() 

In [ ]:
# directory from cornell dataset
cornell=os.path.join(path_file,'data/cornell')

In [ ]:
def get_id2line():
    movie = os.path.join(cornell,'movie_lines.txt')
    lines=open(movie).read().split('\n')
    id2line = {}
    for line in lines:
        _line = line.split(' +++$+++ ')
        if len(_line) == 5:
            id2line[_line[0]] = _line[4]
    return id2line

In [ ]:
def get_conversations():
    conversation = os.path.join(cornell,'movie_conversations.txt')
    conv_lines = open(conversation).read().split('\n')
    convs = [ ]
    for line in conv_lines[:-1]:
        _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
        convs.append(_line.split(','))
    return convs

In [ ]:
def extract_conversations(convs,id2line,path=''):
    idx = 0
    for conv in convs:
        f_conv = open(path + str(idx)+'.txt', 'w')
        for line_id in conv:
            f_conv.write(id2line[line_id])
            f_conv.write('\n')
        f_conv.close()
        idx += 1

In [ ]:
def gather_dataset(convs, id2line):
    questions = []; answers = []

    for conv in convs:
        if len(conv) %2 != 0:
            conv = conv[:-1]
        for i in range(len(conv)):
            if i%2 == 0:
                questions.append(id2line[conv[i]])
            else:
                answers.append(id2line[conv[i]])

    return questions, answers

In [ ]:
def prepare_seq2seq_files(questions, answers, path='',TESTSET_SIZE = 30000):
    train_x = os.path.join(path_file,'processing_data/train.enc')
    train_y = os.path.join(path_file,'processing_data/train.dec')
    test_x = os.path.join(path_file,'processing_data/test.enc')
    test_y = os.path.join(path_file,'processing_data/test.dec')
    # open files
    train_enc = open(train_x,'w')
    train_dec = open(train_y,'w')
    test_enc  = open(test_x, 'w')
    test_dec  = open(test_y, 'w')

    # choose 30,000 (TESTSET_SIZE) items to put into testset
    test_ids = random.sample([i for i in range(len(questions))],TESTSET_SIZE)

    for i in range(len(questions)):
        line = lower_case(questions[i])
        line = rid_number(line)
        line = rid_space(line)
        line = replace_not_dont(line)
        line = rid_character(line)
        line = " ".join(line.split())
        for x in line.split(" "):
            if x not in word_to_index:
                line = line.replace(x, "unk")
        linea = lower_case(answers[i])
        linea = rid_number(linea)
        linea = rid_space(linea)
        linea = replace_not_dont(linea)
        linea = rid_character(linea)
        linea = " ".join(linea.split())
        for x in linea.split(" "):
            if x not in word_to_index:
                linea = linea.replace(x, "unk")
                
        if i in test_ids:
            test_enc.write(line+'\n')
            test_dec.write(linea+ '\n' )
        else:
            train_enc.write(line+'\n')
            train_dec.write(linea+ '\n' )
        if i%2000 == 0:
            print ('written {} lines' .format(str(i)))

    # close files
    train_enc.close()
    train_dec.close()
    test_enc.close()
    test_dec.close()

In [ ]:
id2line = get_id2line()
print ('gathered id2line dictionary.\n')
convs = get_conversations()
print ('>> gathered conversations.\n')
questions, answers = gather_dataset(convs,id2line)

In [ ]:
print ( 'gathered questions and answers.\n')
prepare_seq2seq_files(questions,answers)

In [ ]:
#Normalize all text into lower case
def lower_case(text):
    return text.lower()

In [ ]:
#remove all number
def rid_number(text):
    return re.sub(r'\d+', '', text)

In [ ]:
#remove special characters
def rid_character(text):
    return text.translate(str.maketrans('—’¹“‘”', '      ', string.punctuation))

In [ ]:
#remove white space
def rid_space(text):
    return text.strip()

In [ ]:
#replace negative sentence instance aren't with are not
def replace_not_dont(text):
    if text.find("n't"):
        text = text.replace("n't", " not")
    if text.find("'s"):
        text = text.replace("'s", " is")
    if text.find("'d"):
        text = text.replace("'d", " would")
    if text.find("d've"):
        text = text.replace("d've", " would have")
    if text.find("."):
        text = text.replace(".", " ")
    if text.find(","):
        text = text.replace(",", " ")
    if text.find("'ll"):
        text = text.replace("'ll", " will")
    if text.find("'ve"):
        text = text.replace("'ve", " have")
    return text

In [ ]:
def data():
    processing = os.path.join(path_file,'processing_data/train.dec')
    lines=open(processing).read().split('\n')
    ht = dict()
    conta = 0
    for line in lines:
        '''line = lower_case(line)
        line = rid_number(line)
        line = rid_space(line)
        line = replace_not_dont(line)
        line = rid_character(line)
        _line = " ".join(line.split())'''
        for x in line.split(" "):
            #x = rid_space(x)
            if x not in ht and x!=" ":
                ht[x]=conta
                conta+=1
    return ht

In [ ]:
vocabulary = data()

In [ ]:
#add begin, end and space characters
vocabulary['pad']=3482
vocabulary['bos']=3483
vocabulary['eos']=3484

In [ ]:
n_vocabulary = {v:k for k,v in vocabulary.items()}

In [ ]:
##########################################################################################################

In [ ]:
def load_json(data_file):
    if os.path.isfile(data_file):
        with open(data_file, 'r') as read_file:
            data = json.load(read_file)
            return data

def load_list_file(list_file):
    with open(list_file, 'r') as read_file:
        dialog_id_list = read_file.readlines()
        dialog_id_list = [l.strip('\n') for l in dialog_id_list]
        return dialog_id_list
    return

In [ ]:
dialog_data_file = './data/MultiWOZ/data.json'
dialog_data = load_json(dialog_data_file)
dialog_id_list = list(set(dialog_data.keys())) # Bug: v1.0 contains duplicate id in the valid data
print('# of dialogs:', len(dialog_data))
#print(dialog_data['PMUL4641.json']) # print a sample dialog

In [ ]:
valid_list_file = './data/MultiWOZ/valListFile.json'
test_list_file = './data/MultiWOZ/testListFile.json'

valid_id_list = list(set(load_list_file(valid_list_file)))
test_id_list = load_list_file(test_list_file)
train_id_list = [did for did in dialog_id_list if did not in (valid_id_list + test_id_list)]
print('# of train dialogs:', len(train_id_list))
print('# of valid dialogs:', len(valid_id_list))
print('# of test dialogs :', len(test_id_list))
assert(len(dialog_id_list) == len(train_id_list) + len(valid_id_list) + len(test_id_list))

In [ ]:
train_data = [v for k, v in dialog_data.items() if k in train_id_list]
valid_data = [v for k, v in dialog_data.items() if k in valid_id_list]
test_data = [v for k, v in dialog_data.items() if k in test_id_list]
assert(len(train_data) == len(train_id_list))
assert(len(valid_data) == len(valid_id_list))
assert(len(test_data) == len(test_id_list))

In [ ]:
def get_dst_diff(prev_d, crnt_d):
    assert len(prev_d) == len(crnt_d)
    diff = {}
    for ((k1, v1), (k2, v2)) in zip(prev_d.items(), crnt_d.items()):
        assert k1 == k2
        if v1 != v2: # updated
            diff[k2] = v2
    return diff

def analyze_dialog(train_data, print_dialog=True):
    question = []
    answer = []
    for d in train_data:
        if print_dialog:
            prev_d = None
            for i, t in enumerate(d['log']):
                spk = 'Usr' if i % 2 == 0 else 'Sys' # Turn 0 is always a user's turn in this corpus.
                if spk == 'Sys':
                    if prev_d is None:
                        prev_d = t['metadata']
                    else:
                        crnt_d = t['metadata']
                        dst_diff = get_dst_diff(prev_d, crnt_d)
                        #print('Updated DST:', dst_diff)
                        prev_d = crnt_d
                if i % 2 == 0:
                    u = t['text']
                    question.append(u)
                else:
                    u = t['text']
                    answer.append(u)
                #print('{}: {}'.format(spk, u))
    return question, answer


In [ ]:
q,a = analyze_dialog(test_data, True)
prepare_seq2seq_files(q,a,TESTSET_SIZE = 0)

In [ ]:
def get_train_valid():
    x = os.path.join(path_file,'processing_data/train.enc')
    y = os.path.join(path_file,'processing_data/train.dec')
    x_ = os.path.join(path_file,'processing_data/valid.enc')
    y_ = os.path.join(path_file,'processing_data/valid.dec')
    X = []
    Y = []
    X_ = []
    Y_ = []
    lines=open(x).read().split('\n')
    for line in lines:
        X.append("bos " +line+" eos")
    lines=open(y).read().split('\n')
    for line in lines:
        Y.append("bos " +line+" eos")
    lines=open(x_).read().split('\n')
    for line in lines:
        X_.append("bos " +line+" eos")
    lines=open(y_).read().split('\n')
    for line in lines:
        Y_.append("bos " +line+" eos")
    return np.array(X),np.array(Y),np.array(X_),np.array(Y_)

In [ ]:
X,Y,X_,Y_=get_train_valid()

In [ ]:
hdf5_path=os.path.join(path_file,'peso/2.h5')
file = h5py.File(hdf5_path, mode='r+')
print(list(file.keys()))
for a in file['embedding_1']:
    print(a)
file.close()

# Data

In [ ]:
#create dataset data.h5
hdf5_path=os.path.join(path_file,'data_h5/data.h5')

In [ ]:
file=h5py.File(hdf5_path,'w')
x_r=file.create_group('X')
y_r=file.create_group('Y')
x_v=file.create_group('X_')
y_v=file.create_group('Y_')
yh_ta=file.create_group('Yh_train')
yh_te=file.create_group('Yh_test') 
file.close()

In [ ]:
file = h5py.File(hdf5_path, mode='r+')
dataset=file['/Yh_train'].create_dataset('yh_train',(56779,63,3485), np.int8)
#dataset=file['/Yh_test'].create_dataset('yh_test',(7375,63,50), np.int8)
c = 0
for x in range(50,56779,50): 
    dataset[c:x] = get_hot(Y[c:x], Ty = 63)
    c=x
dataset=file['/Yh_test'].create_dataset('yh_test',(7375,63,3485), np.int8)
c = 0
for x in range(50,7375,50): 
    dataset[c:x] = get_hot(Y_[c:x], Ty = 63)
    c=x
    
dataset=file['/X'].create_dataset('x',X_train.shape, np.float64)
dataset[...]=X_train

dataset=file['/Y'].create_dataset('y',Y_train.shape, np.float64)
dataset[...]=Y_train

dataset=file['/X_'].create_dataset('x_',X_test.shape, np.float64)
dataset[...]=X_test

dataset=file['/Y_'].create_dataset('y_',Y_test.shape, np.float64)
dataset[...]=Y_test

file.close()

In [ ]:
for i in Y:
    l = len(i.split())
    if  l > 63:
        print(i)

In [ ]:
maxLenX = len(max(X, key=len).split())
maxLenY = len(max(Y, key=len).split())
maxLenX_ = len(max(X_, key=len).split())
maxLenY_ = len(max(Y_, key=len).split())

In [ ]:
print("{} {} {} {}".format(maxLenX,maxLenY,maxLenX_,maxLenY_))

In [ ]:
def get_hot(n, Ty):
    Y=[]
    vocab=n
    for i in vocab:
        string=i.split(' ')
        rep = list(map(lambda x: vocabulary.get(x, 57), string))
        if len(string) < Ty:
            rep += [vocabulary['pad']] * (Ty - len(string))
        Y.append(rep)
    return np.array(list(map(lambda x: to_categorical(x, num_classes=3485), Y)))

In [ ]:
Y_oh_train = get_hot(Y[:100], Ty = 63)
Y_oh_test = get_hot(Y_[:200], Ty = 63)

In [ ]:
def read_glove_vecs(glove_file):
    with open(glove_file,encoding="utf-8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [ ]:
embedings = os.path.join(path_file,'worc2vec/glove.6B.50d.txt')
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs(embedings)

In [ ]:
print(index_to_word[138209.])

In [ ]:
contador = 0
for x in file:
    if x not in word_to_index:
        print(x)
        contador+= 1
print(contador)

In [ ]:
contador = 0
for x in file:
    if x not in word_to_index:
        print(x)
        contador+= 1
print(contador)

In [ ]:
print(word_to_index['pad'])

In [ ]:
path_dataset = os.path.join(path_file,'data_h5/data.h5')
dataset = h5py.File(path_dataset, "r")
x=  np.array(dataset["/X/x"][6])
y=  np.array(dataset["/Y/y"][1])
yh= np.array(dataset["/Yh_train/yh_train"][1])
dataset.close()

In [ ]:
a = yh.argmax(axis=-1)
letra = list(map(lambda x:n_vocabulary[x],a))
print(yh.argmax(axis=-1))
print(x)
print(index_to_word[185457.])

In [ ]:
#################################################################################################

In [ ]:
def getDataTrain(batch_size):
    dataset = h5py.File(path_dataset, "r")
    data_num=len(dataset['/X/x'])
    batches_list = list(range(int(mt.ceil(float(data_num) / batch_size))))
    while True:
        for n, i in enumerate(batches_list):
            i_s = i * batch_size  # index of the first image in this batch
            i_e = min([(i + 1) * batch_size, data_num])  # index of the last image in this batch
            x=  np.array(dataset["/X/x"][i_s: i_e])
            y=  np.array(dataset["/Y/y"][i_s: i_e])
            yh= np.array(dataset["/Yh_train/yh_train"][i_s: i_e])
            yield [x,y], yh  
    dataset.close()

In [ ]:
def getDataVal(batch_size):
    dataset = h5py.File(path_dataset, "r")
    data_num=len(dataset['/X_/x_'])
    batches_list = list(range(int(mt.ceil(float(data_num) / batch_size))))
    while True:
        # loop over batches
        for n, i in enumerate(batches_list):
            i_s = i * batch_size  # index of the first image in this batch
            i_e = min([(i + 1) * batch_size, data_num])  # index of the last image in this batch
            x=  np.array(dataset["/X_/x_"][i_s: i_e])
            y=  np.array(dataset["/Y_/y_"][i_s: i_e])
            yh= np.array(dataset["/Yh_test/yh_test"][i_s: i_e])
            yield [x,y], yh
    dataset.close()

In [ ]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =X[i].lower().split()
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j+1
            
    ### END CODE HERE ###
    
    return X_indices

In [ ]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len,emb_dim,trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
X_train = sentences_to_indices(X, word_to_index, 63)
Y_train = sentences_to_indices(Y, word_to_index, 63)
X_test = sentences_to_indices(X_, word_to_index, 63)
Y_test = sentences_to_indices(Y_, word_to_index, 63)

In [ ]:
print(X_train[0])

In [ ]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

In [ ]:
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

In [ ]:
def embed(word_to_vec_map, 
              word_to_index,
              step_input=63):
    sentence_indices = Input(shape=(step_input,), dtype='int32')
     # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    embed = Model(inputs = [sentence_indices], outputs = embeddings)
    ### END CODE HERE ###
        
    return embed

In [ ]:
embed = embed(word_to_vec_map, word_to_index)

In [ ]:
aggg = embed.predict(Y_train[:1])

In [ ]:
Y_test = embed.predict(Y_test)

# Model

In [ ]:
def model( word_to_vec_map, 
              word_to_index,
              step_input=63,
              step_out=63,
              size_input_r=50,
              output_dim=3485,
              atten_units=64,
              encoder_units=120,
              decoder_units=240,
              gmax= 60,
              trainable=True,
              return_probabilities=False):
    """
    Arguments:
    step_input -- length of the input sequence
    step_out-- length of the output sequence
    size_input_r -- dim input RGB image (CNN output)
    size_input_d -- dim input depth image (CNN output)
    size_input_j -- dim input skeleton image (CNN output)
    output_dim -- size of the python dictionary "lsp"
    atten_units -- number of units in attention (dense)
    encoder_units -- hidden state in encoder
    decoder_units -- hidden state in decoder
    trainable -- boolean trainable or not trainable
    return_probabilities -- boolean return attention probabilities
    
    Returns:
    model -- Keras model instance
    """ 
    """
    Builds a Neural Machine Translator that has alignment attention
    
    :return: keras.models.Model that can be compiled and fit'ed
    *** REFERENCES ***
    Lee, Jason, Kyunghyun Cho, and Thomas Hofmann. 
    "Neural Machine Translation By Jointly Learning To Align and Translate
    " 
    """
            
    sentence_indices_question = Input(shape=(step_input,), dtype='int32')
     # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings_q = embedding_layer(sentence_indices_question)#(m, 40, 50)
    # Step 2: encoder with two LSTM layers.
    shortcut = Bidirectional(LSTM(encoder_units, return_sequences=True, dropout=0.5, recurrent_dropout=0.5),merge_mode='concat',trainable=trainable, name='BLSTM')(embeddings_q)
    a= Bidirectional(LSTM(encoder_units, return_sequences=True, dropout=0.5, recurrent_dropout=0.5),merge_mode='concat',trainable=trainable, name='R_BLSTM')(shortcut)
    a = Dropout(0.5, name='DROP_1')(a)
    a = Add(name = 'ADD_1')([a,shortcut])
    # Step 3: attention decoder.
    sentence_indices_answer = Input(shape=(step_out,), dtype='int32')
     # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings_a = embedding_layer(sentence_indices_answer)#(m, 63, 50)
    
    y_hat = attention_LSTM(decoder_units,
                             steps=step_out,
                             output_dim=output_dim,
                             atten_units = atten_units,
                             gmax = gmax,
                             return_sequences=True,
                             return_probabilities=return_probabilities,
                             trainable=trainable,
                             name = 'ATT_DEC')([a,embeddings_a])
    
    # Step 4: Create model instance taking three inputs and returning the list of outputs.
    model = Model(inputs = [sentence_indices_question, sentence_indices_answer], outputs = y_hat)
    ### END CODE HERE ###
        
    return model

In [ ]:
# define model
model = model(word_to_vec_map, word_to_index)

In [ ]:
# summary model
model.summary()

In [ ]:
# plot model seq to seq (LSTM) with attention
plot_model(model, to_file='chat.png', show_shapes=True)

In [ ]:
# compile model
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.01) # rgbdj
#opt = RMSprop(lr=0.045, rho=0.94, epsilon=1.0, decay=0) #rgb
early_stopping=EarlyStopping(monitor='val_loss', patience=6)
mcp_save = ModelCheckpoint('chat.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')

model.compile(optimizer=opt, metrics=['accuracy'], loss='categorical_crossentropy')

In [ ]:
#X_j = np.expand_dims(X_j, axis=1)
#X_j_ = np.expand_dims(X_j_, axis=1)
# train model
History=model.fit([X_train[:5000], Y_train[:5000]], Y_oh_train, epochs=2000,  callbacks=[early_stopping,mcp_save], batch_size=125, validation_data=([X_test[:200],Y_test[:200]],Y_oh_test))

In [ ]:
path_dataset = os.path.join(path_file,'data_h5/data.h5')
tr_gen =  getDataTrain( batch_size=100 )
va_gen =  getDataVal( batch_size=74 )

In [ ]:
History=model.fit_generator(tr_gen, steps_per_epoch=568, epochs=1024, verbose=1,  callbacks=[early_stopping,mcp_save],  validation_data=va_gen, validation_steps=100)

In [ ]:
# plot train and validation loss
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper right')
plt.show()

In [ ]:
# plot train and validation acc
plt.plot(History.history['acc'])
plt.plot(History.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['acc', 'val_acc'], loc='best')
plt.show()

In [ ]:
#save training history 
import pickle
with open('chat_bot.pickle', 'wb') as file_pi:
        pickle.dump(History.history, file_pi, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# plot train and validation loss
#plt.figure(figsize=(14, 10))
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model - Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper right')
plt.savefig(os.path.join(path_file,'validation_loss.png'), bbox_inches='tight')

In [ ]:
plt.plot(History.history['acc'])
plt.plot(History.history['val_acc'])
plt.title('Model - Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')
plt.savefig(os.path.join(path_file,'validation_acc.png'), bbox_inches='tight')

In [ ]:
model.save_weights('chat_bot_wight.h5')

In [ ]:
WEIGHTS_PATH = os.path.join(path_file,'peso/2.h5')
model.load_weights(WEIGHTS_PATH, by_name=True)

In [ ]:
aa = embedding_layer.get_weights()
print(np.array(aa).shape)

In [ ]:
model.layers[1].set_weights(aa)

In [ ]:
def predict( sent ):
    words = sent.split(' ')
    words = ['bos'] + words + ['eos']
    words_id = []
    for w in words:
        if w in word_to_index:
            words_id.append( word_to_index[w] )
        else:
            words_id.append( word_to_index['unk'] )
    words = words_id


    ret = ""
    print(words)
    m_input = [ np.zeros((1,63)) , np.zeros((1,63)) ]
    for i , w in enumerate( words ):
        m_input[0][0 , i ] = w
    m_input[1][0,0] = word_to_index['bos']
    for w_i in range(1,63):
        out = model.predict( m_input )
        
        out_w_i = np.argmax(out[0][w_i-1])
        if out_w_i == 0:
            continue
        ret +=  n_vocabulary[out_w_i] + " "
        out_w_i = word_to_index[n_vocabulary[out_w_i]]
        m_input[1][0,w_i] = out_w_i

    return ret

In [ ]:
while True:
    sent = input("Enter a sentence : ")
    print (predict( sent ).encode('utf-8'))

    print ("="*30)